## Prepare

In [ ]:
import os
import json
import ipywidgets as widgets
from ipywidgets import Text, HTML, HBox, Layout, VBox
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    WidgetControl,
    basemaps,
    Popup,
    Icon,
    MarkerCluster
)

from math import sqrt

In [ ]:
from fabrictestbed.slice_manager import SliceManager, Status

#Retrieve or set the refresh token (exprires 24 hours after login)
#fabric_refresh_token=None
#%store -r fabric_refresh_token
#if fabric_refresh_token is None:
    #fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    #%store fabric_refresh_token

#print("Refresh Token: {}".format(fabric_refresh_token))

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credmgr_host, project_name='all', scope='all')
slice_manager.initialize()

print("Token location: {}".format(slice_manager.token_location))
print()
print("Tokens initialized: {}".format(slice_manager.tokens))

#Cofigure SSH Key
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

    
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()

In [ ]:
site_dict = {'UKY': (38.03154264461649, -84.50343326019218), 'RENCI': (35.93996154497003, -79.01809264307425), 'LBNL': (38.94977311613871, -122.61289106281062)}

## Create experiment with map version 1

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer

# Create Experiment Topology
experiment = ExperimentTopology()

slice_name = "MapDemoSlice"

## Node functions

In [ ]:
node_name = widgets.Text(
    placeholder='enter name',
    description='Node Name:',
    disabled=False
)

site_select = widgets.Select(
    options=['RENC', 'UKY', 'LBNL'],
    value='UKY',
    description='Site:',
    disabled=False
)

site_option = widgets.Text(
    value='UKY',
    description='Site:',
    disabled=False
)

core_slider = widgets.IntSlider(min=2, max=32, step=2, value=4, description='Cores')
ram_slider = widgets.IntSlider(min=2, max=64, step=2, value=16, description='RAM')
disk_slider = widgets.FloatLogSlider(base=2, min=5, max=17, step=1, value=128,description='Disk')

img_ref_select = widgets.Select(
    options=['qcow2', 'qchicken2', 'qpig2'],
    value='qcow2',
    description='Image Type:',
    disabled=False
)

img_type_select = widgets.Select(
    options=['default_ubuntu_20', 'default_arch_20', 'default_kali_21'],
    value='default_ubuntu_20',
    description='Image Reference:',
    disabled=False
)

In [ ]:
from ipyleaflet import DrawControl, CircleMarker

canvas = Map(basemap = basemaps.Esri.WorldStreetMap, center=(38.12480976137421, -95.7129), zoom = 3.8)

dc = DrawControl(circlemarker={}, polygon={})


init = [HTML('Click on a site to create a new node')]
my_layout = Layout(height='50%', overflow_y= 'auto', padding = "4px" )
html_box = VBox(init , layout = my_layout )
control = WidgetControl(widget = html_box, position='topright')
canvas.add_control(control)
  
canvas.add_control(dc)

uky_coordinates = site_dict['UKY']
renci_coordinates = site_dict['RENCI']
lbnl_coordinates = site_dict['LBNL']

icon = Icon(icon_url='https://jupyter-beta.fabric-testbed.net/user/nannkat@bu.edu/files/jupyter-examples/nanna/map_experiments/circle.png?_xsrf=2%7C4b270639%7C01ca1ecbc98a0090a488f8688b3a0dd1%7C1625580900', icon_size=[15,15])

uky_circle = Marker(location = uky_coordinates, icon = icon, draggable = False, title = 'UKY')
renci_circle = Marker(location = renci_coordinates, icon = icon, draggable = False, title = 'RENCI')
lbnl_circle = Marker(location = lbnl_coordinates, icon = icon, draggable = False, title = 'LBNL')

marker_lbnl = Marker(location = lbnl_coordinates, draggable = False)
marker_uky = Marker(location = uky_coordinates, draggable = False)
marker_renci = Marker(location = renci_coordinates, draggable = False)

def add_node(self):
    n1 = experiment.add_node(name=node_name.value, site=site_option.value)
    cap = Capacities().set_fields(core=core_slider.value, ram=ram_slider.value, disk=disk_slider.value)
    n1.set_properties(capacities=cap)
    node_name.value = ''
    html_box.children = init


add_node_request = widgets.Button(
    description='Add Node',
    disabled=False,
    tooltip='click to build node',
)

add_node_request.on_click(add_node)

def esc(self):
    html_box.children = init
    node_name.value = ''
    if site_option.value == 'LBNL':
        canvas.remove_layer(marker_lbnl)
    elif site_option.value == 'UKY':
        canvas.remove_layer(marker_uky)
    elif site_option.value == 'RENCI':
        canvas.remove_layer(marker_renci)
    
exit_btn = widgets.Button(
    description='Cancel',
    disabled=False,
    tooltip='click to exit',)

exit_btn.on_click(esc)


def node_dashboard(site_name):
    header = HTML('<center><b>New Node</b></center>')
    site_option.value = site_name
    btns = HBox([VBox([add_node_request]), VBox([exit_btn])])
    content = [header,node_name, site_option, core_slider, ram_slider, disk_slider, btns]
    html_box.children = content

def new_lbnl(**kwargs):
    canvas.add_layer(marker_lbnl)
    node_dashboard('LBNL')
    
def new_uky(**kwargs):
    canvas.add_layer(marker_uky)
    node_dashboard('UKY')
    
def new_renci(**kwargs):
    canvas.add_layer(marker_renci)
    node_dashboard('RENCI')
    

def remove_uky(**kwargs):
    canvas.remove_layer(marker_uky)
    
def remove_lbnl(**kwargs):
    canvas.remove_layer(marker_lbnl)
    
def remove_renci(**kwargs):
    canvas.remove_layer(marker_renci)
    
marker_uky.on_click(remove_uky)
marker_lbnl.on_click(remove_lbnl)
marker_renci.on_click(remove_renci)

lbnl_circle.on_click(new_lbnl)
uky_circle.on_click(new_uky)
renci_circle.on_click(new_renci)

clussss = MarkerCluster(markers = (uky_circle, renci_circle, lbnl_circle))
canvas.add_layer(uky_circle)
canvas.add_layer(renci_circle)
canvas.add_layer(lbnl_circle)

#canvas.add_layer(clussss)
canvas

In [ ]:
print(experiment.nodes)  